In [1]:
%load_ext autoreload
%autoreload 2

#### Provides an example of using an AssessmentDataDownloader

In [8]:
from deh.dl.squad import SquadAssessmentDataDownloader

# Instantiate a SquadAssessmentDataDownloader:
dl = SquadAssessmentDataDownloader(cache_dir="../../data/qa_dl_cache")

In [7]:
dl.save_contexts( "../../data/contexts/" )
dl.save_question_answers( "../../data/qas/qas.tsv" )